In [1]:
#import packages
import sdRDM
from sdRDM import DataModel
from sdRDM.database import build_sql_database

In [2]:
#import the data base
pyeed = DataModel.from_git("https://github.com/maxim945/PyEED-Fullprototype.git")
#visualize for each separate table
pyeed.ProteinSequence.visualize_tree()
pyeed.DNASequence.visualize_tree()
pyeed.Organism.visualize_tree()

ProteinSequence
├── id
├── protein_sequence_id
├── name
├── amino_acid_sequence
├── protein_database_name
│   └── ProteinDatabase
│       ├── id
│       ├── database
│       └── link_to_database
└── protein_organism_id
    └── Organism
        ├── id
        ├── organism_name
        └── ncbi_taxonomy_id
DNASequence
├── id
├── protein_sequence_id
│   └── ProteinSequence
│       ├── id
│       ├── protein_sequence_id
│       ├── name
│       ├── amino_acid_sequence
│       ├── protein_database_name
│       │   └── ProteinDatabase
│       │       ├── id
│       │       ├── database
│       │       └── link_to_database
│       └── protein_organism_id
│           └── Organism
│               ├── id
│               ├── organism_name
│               └── ncbi_taxonomy_id
├── dna_sequence_id
├── nucleotide_sequence
├── dna_database_id
│   └── DNADatabase
│       ├── id
│       ├── database
│       └── link_to_database
└── dna_organism_id
    └── Organism
        ├── id
        ├── organism_nam

In [3]:
#build the data base
build_sql_database(pyeed.ProteinSequence, loc="./Cluster.db")
build_sql_database(pyeed.DNASequence, loc="./Cluster.db")
build_sql_database(pyeed.Organism, loc="./Cluster.db")

In [7]:
# query protein sequence input
from Bio import SeqIO
protein_seq = SeqIO.read("TEM-1", "fasta")

In [8]:
# run an online blast search in the NCBI database
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("tblastn", "nt", protein_seq.seq,expect=0.0000001,
                               hitlist_size=100,threshold = 550) 

In [9]:
# save the ncbi search results as an xml document
with open("TEM-1nt.xml", "w") as save_to:
    save_to.write(result_handle.read())
    result_handle.close()

In [10]:
import xml.etree.ElementTree as ET
import pandas as pd

# chose your headers, which columns you are interested in, in your database.
cols = ["dna_sequence_id"]
rows = []


# PARSE XML
tree = ET.parse('TEM-1nt.xml')
root = tree.getroot()

# search for the specific hits in the xml file
for child in root.iter():

    if child.tag == 'Hit_accession':
        Hit_accession=child.text.strip()

# append data of interest into the database        
        rows.append({"dna_sequence_id": Hit_accession})
# create a dataframe and visualize it
dna_sequence_iddf = pd.DataFrame(rows, columns=cols)
print(dna_sequence_iddf) 

   dna_sequence_id
0         MT387486
1         MT387485
2         MT387484
3         MT387483
4         MT387482
..             ...
95        MZ026128
96        MZ026126
97        MZ026125
98        MZ026124
99        MZ026123

[100 rows x 1 columns]


In [11]:
from Bio import Entrez

cols = ["protein_sequence_id"]
rows = []

Entrez.email = "youremail@example.com" # replace with your email
for index, row in dna_sequence_iddf.iterrows():
    handle = Entrez.efetch(db="nucleotide", id=row["dna_sequence_id"], rettype="gb", retmode="text")
    record = handle.read()
    #record = SeqIO.read(handle, "genbank")

    #print(record)
    for line in record.splitlines():
        if line.startswith("                     /protein_id="):
            protein_id = line.strip().split("=")[1].strip('"')
            #print(protein_id)
            
            rows.append({"protein_sequence_id": protein_id})
# create a dataframe and visualize it
protein_sequence_iddf = pd.DataFrame(rows, columns=cols)
print(protein_sequence_iddf) 

   protein_sequence_id
0           QJE70211.1
1           QJE70210.1
2           QJE70209.1
3           QJE70208.1
4           QJE70207.1
..                 ...
95          QWY17573.1
96          QWY17571.1
97          QWY17570.1
98          QWY17569.1
99          QWY17568.1

[100 rows x 1 columns]


In [12]:
from Bio import Entrez

cols = ["protein_sequence_id_id"]
rows = []

Entrez.email = "youremail@example.com" # replace with your email
for index, row in dna_sequence_iddf.iterrows():
    handle = Entrez.efetch(db="nucleotide", id=row["dna_sequence_id"], rettype="gb", retmode="text")
    record = handle.read()
    #record = SeqIO.read(handle, "genbank")

    #print(record)
    for line in record.splitlines():
        if line.startswith("                     /protein_id="):
            protein_id = line.strip().split("=")[1].strip('"')
            #print(protein_id)
            
            rows.append({"protein_sequence_id_id": protein_id})
# create a dataframe and visualize it
protein_sequence_id_iddf = pd.DataFrame(rows, columns=cols)
print(protein_sequence_id_iddf) 

   protein_sequence_id_id
0              QJE70211.1
1              QJE70210.1
2              QJE70209.1
3              QJE70208.1
4              QJE70207.1
..                    ...
95             QWY17573.1
96             QWY17571.1
97             QWY17570.1
98             QWY17569.1
99             QWY17568.1

[100 rows x 1 columns]


In [13]:
from Bio import Entrez

cols = ["organism_name"]
rows = []

Entrez.email = "youremail@example.com" # replace with your email
for index, row in dna_sequence_iddf.iterrows():
    handle = Entrez.efetch(db="nucleotide", id=row["dna_sequence_id"], rettype="gb", retmode="text")
    record = handle.read()
    #record = SeqIO.read(handle, "genbank")

    #print(record)
    for line in record.splitlines():
        if line.startswith("                     /organism="):
            organism_id = line.strip().split("=")[1].strip('"')
            #print(protein_id)
            
            rows.append({"organism_name": organism_id})
# create a dataframe and visualize it
organism_iddf = pd.DataFrame(rows, columns=cols)
print(organism_iddf) 

       organism_name
0   Escherichia coli
1   Escherichia coli
2   Escherichia coli
3   Escherichia coli
4   Escherichia coli
..               ...
95  Escherichia coli
96  Escherichia coli
97  Escherichia coli
98  Escherichia coli
99  Escherichia coli

[100 rows x 1 columns]


In [14]:
from Bio import Entrez

cols = ["protein_organism_id_id"]
rows = []

Entrez.email = "youremail@example.com" # replace with your email
for index, row in dna_sequence_iddf.iterrows():
    handle = Entrez.efetch(db="nucleotide", id=row["dna_sequence_id"], rettype="gb", retmode="text")
    record = handle.read()
    #record = SeqIO.read(handle, "genbank")

    #print(record)
    for line in record.splitlines():
        if line.startswith("                     /db_xref="):
            taxon_value = line.strip().split("=")[1].strip('"')
            #print(taxon_value)
            if taxon_value.startswith("taxon:"):
                organism_id = taxon_value.split(":")[1]
                #print(organism_id)
            
                rows.append({"protein_organism_id_id": organism_id})
# create a dataframe and visualize it
protein_organism_iddf = pd.DataFrame(rows, columns=cols)
print(protein_organism_iddf) 

   protein_organism_id_id
0                     562
1                     562
2                     562
3                     562
4                     562
..                    ...
95                    562
96                    562
97                    562
98                    562
99                    562

[100 rows x 1 columns]


In [15]:
from Bio import Entrez

cols = ["dna_organism_id_id"]
rows = []

Entrez.email = "youremail@example.com" # replace with your email
for index, row in dna_sequence_iddf.iterrows():
    handle = Entrez.efetch(db="nucleotide", id=row["dna_sequence_id"], rettype="gb", retmode="text")
    record = handle.read()
    #record = SeqIO.read(handle, "genbank")

    #print(record)
    for line in record.splitlines():
        if line.startswith("                     /db_xref="):
            taxon_value = line.strip().split("=")[1].strip('"')
            #print(taxon_value)
            if taxon_value.startswith("taxon:"):
                organism_id = taxon_value.split(":")[1]
                #print(organism_id)
            
                rows.append({"dna_organism_id_id": organism_id})
# create a dataframe and visualize it
dna_organism_iddf = pd.DataFrame(rows, columns=cols)
print(dna_organism_iddf) 

   dna_organism_id_id
0                 562
1                 562
2                 562
3                 562
4                 562
..                ...
95                562
96                562
97                562
98                562
99                562

[100 rows x 1 columns]


In [16]:
from Bio import Entrez

cols = ["ncbi_taxonomy_id"]
rows = []

Entrez.email = "youremail@example.com" # replace with your email
for index, row in dna_sequence_iddf.iterrows():
    handle = Entrez.efetch(db="nucleotide", id=row["dna_sequence_id"], rettype="gb", retmode="text")
    record = handle.read()
    #record = SeqIO.read(handle, "genbank")

    #print(record)
    for line in record.splitlines():
        if line.startswith("                     /db_xref="):
            taxon_value = line.strip().split("=")[1].strip('"')
            #print(taxon_value)
            if taxon_value.startswith("taxon:"):
                organism_id = taxon_value.split(":")[1]
                #print(organism_id)
            
                rows.append({"ncbi_taxonomy_id": organism_id})
# create a dataframe and visualize it
ncbi_taxonomy_iddf = pd.DataFrame(rows, columns=cols)
print(ncbi_taxonomy_iddf) 

   ncbi_taxonomy_id
0               562
1               562
2               562
3               562
4               562
..              ...
95              562
96              562
97              562
98              562
99              562

[100 rows x 1 columns]


In [17]:
#protein_database_name

import pandas as pd

# Create a dataframe with 50 rows and 1 column
protein_database_namedf = pd.DataFrame(["GeneBank"]*100, columns=["protein_database_name_id"])

In [18]:
#protein_database_name

import pandas as pd

# Create a dataframe with 50 rows and 1 column
dna_database_namedf = pd.DataFrame(["GeneBank"]*100, columns=["dna_database_id_id"])

In [19]:
#protein_database_name

import pandas as pd

# Create a dataframe with 50 rows and 1 column
namedf = pd.DataFrame(["TEM-1"]*100, columns=["name"])

In [20]:
import xml.etree.ElementTree as ET
import pandas as pd

# chose your headers, which columns you are interested in, in your database.
cols = ["amino_acid_sequence"]
rows = []


# PARSE XML
tree = ET.parse('TEM-1nt.xml')
root = tree.getroot()

# search for the specific hits in the xml file
for child in root.iter():

    if child.tag == 'Hsp_hseq':
        Hsp_hseq=child.text.strip()

# append data of interest into the database        
        rows.append({"amino_acid_sequence": Hsp_hseq})
# create a dataframe and visualize it
amino_acid_sequencedf = pd.DataFrame(rows, columns=cols)
print(amino_acid_sequencedf) 

                                  amino_acid_sequence
0   HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
1   HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
2   HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
3   HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
4   HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
..                                                ...
95  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
96  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
97  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
98  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
99  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...

[100 rows x 1 columns]


In [21]:
# convert the xml to a dataframe

import xml.etree.ElementTree as ET
import pandas as pd
from Bio import Entrez

# chose your headers, which columns you are interested in, in your database.
cols = ["nucleotide_sequence"]
rows = []


# PARSE XML
tree = ET.parse('TEM-1nt.xml')
root = tree.getroot()

# search for the specific hits in the xml file
for child in root.iter():
    if child.tag == 'Hit_accession':
        Hit_accession=child.text.strip()
        Entrez.email = "maximschwarzberg@gmail.com" # Always tell NCBI who you are
        handle = Entrez.efetch(db="nucleotide", id=Hit_accession, rettype="fasta")
        dna_sequence = handle.read()
        dna_sequence = dna_sequence.split('\n')[1:]
        dna_sequence = ''.join(dna_sequence)
        #print(dna_sequence)
        rows.append({"nucleotide_sequence": dna_sequence})
# create a dataframe and visualize it
nucleotide_sequencedf = pd.DataFrame(rows, columns=cols)
print(nucleotide_sequencedf) 
        #with open("accessionnt.txt", "a") as f:
            #print(dna_sequence, file=f)
            


                                  nucleotide_sequence
0   ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
1   ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2   ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
3   ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4   ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
..                                                ...
95  ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
96  ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
97  ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
98  ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
99  ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...

[100 rows x 1 columns]


In [22]:
import sqlite3

# Connecting to the database
connection = sqlite3.connect('Cluster.db')

# inserting all dataframes to a one merged dataframe on the same index
merged_df = pd.concat([protein_sequence_iddf, namedf, amino_acid_sequencedf,
                       protein_database_namedf,protein_organism_iddf], axis=1)
#print(merged_df)
# Insert the dataframe into the database ProteinSequence table
merged_df.to_sql('ProteinSequence', connection, if_exists='append', index=False)

# Committing the changes
connection.commit()
 
# closing the database connection
connection.close()

In [23]:
import sqlite3

# Connecting to the database
connection = sqlite3.connect('Cluster.db')

# inserting all dataframes to a one merged dataframe on the same index
merged_df = pd.concat([protein_sequence_id_iddf, dna_sequence_iddf, nucleotide_sequencedf,
                       dna_database_namedf,dna_organism_iddf], axis=1)
#print(merged_df)
# Insert the dataframe into the database ProteinSequence table
merged_df.to_sql('DNASequence', connection, if_exists='append', index=False)

# Committing the changes
connection.commit()
 
# closing the database connection
connection.close()

In [24]:
import sqlite3

# Connecting to the database
connection = sqlite3.connect('Cluster.db')

# inserting all dataframes to a one merged dataframe on the same index
merged_df = pd.concat([organism_iddf, ncbi_taxonomy_iddf], axis=1)
#print(merged_df)
# Insert the dataframe into the database ProteinSequence table
merged_df.to_sql('Organism', connection, if_exists='append', index=False)

# Committing the changes
connection.commit()
 
# closing the database connection
connection.close()